In [8]:
import pandas as pd
import datetime
import numpy as np

data_noise_hour = pd.read_csv("../Data/hourly_noisedata_2022.csv")
print(data_noise_hour.head())

print(len(data_noise_hour))

   month  day  hour                       description      lamax       laeq
0      1    1     0      MP 03: Naamsestraat 62 Taste  60.322528  57.126833
1      1    1     0    MP 05: Calvariekapel KU Leuven  53.230972  49.987639
2      1    1     0  MP 06: Parkstraat 2 La Filosovia  53.666056  50.752000
3      1    1     0            MP 07: Naamsestraat 81  50.056861  47.440222
4      1    1     1      MP 03: Naamsestraat 62 Taste  53.033583  50.853806
55469


In [7]:
data_weather_hour = pd.read_csv("../Data/hourly_weatherdata_2022.csv")
print(data_weather_hour.head())

print(len(data_weather_hour))

   Month  Day  Hour  LC_RAININ  LC_DAILYRAIN  LC_WINDDIR  LC_WINDSPEED   
0      1    1     0   0.000003       0.00036  -18.197324      0.389565  \
1      1    1     1   0.000007       0.00000  -16.227891      0.222602   
2      1    1     2   0.000009       0.00000  -13.710884      0.217194   
3      1    1     3   0.000000       0.00000  -16.401361      0.178248   
4      1    1     4   0.000000       0.00000  -10.268707      0.237670   

   LC_TEMP_QCL3  
0     13.100358  
1     12.669197  
2     12.520271  
3     12.386194  
4     12.080706  
8760


In [13]:
# Select the columns of interest from data_noise_hour
noise_columns = ["month", "day", "hour"]

# Select the columns of interest from data_weather_hour
weather_columns = ["Month", "Day", "Hour", "LC_RAININ", "LC_DAILYRAIN", "LC_WINDDIR", "LC_WINDSPEED"]

# Merge the dataframes based on the common columns
data_model = data_noise_hour.merge(data_weather_hour[weather_columns], left_on=noise_columns, right_on=weather_columns[:-4], how='left')

#print(data_model.head(20))

# Remove the columns "Month", "Day", and "Hour" from data_model
data_model = data_model.drop(["Month", "Day", "Hour"], axis=1)

print(data_model.head(20))

    month  day  hour                       description      lamax       laeq   
0       1    1     0      MP 03: Naamsestraat 62 Taste  60.322528  57.126833  \
1       1    1     0    MP 05: Calvariekapel KU Leuven  53.230972  49.987639   
2       1    1     0  MP 06: Parkstraat 2 La Filosovia  53.666056  50.752000   
3       1    1     0            MP 07: Naamsestraat 81  50.056861  47.440222   
4       1    1     1      MP 03: Naamsestraat 62 Taste  53.033583  50.853806   
5       1    1     1    MP 05: Calvariekapel KU Leuven  53.599639  50.578806   
6       1    1     1  MP 06: Parkstraat 2 La Filosovia  59.880694  56.942167   
7       1    1     1            MP 07: Naamsestraat 81  50.097278  47.878333   
8       1    1     2      MP 03: Naamsestraat 62 Taste  52.173702  50.049903   
9       1    1     2    MP 05: Calvariekapel KU Leuven  51.078083  47.974361   
10      1    1     2  MP 06: Parkstraat 2 La Filosovia  52.107194  48.968806   
11      1    1     2            MP 07: N

In [15]:
# Remove the columns "Month", "Day", and "Hour" from data_model
data_model = data_model.drop(["LC_RAININ"], axis=1)

#wind direction could have an impact as it "carries" the noise, could bring the noise to another place
#e.g. noise from the center is carried by the wind to the Naamse straat

In [ ]:
#transform the variables month day and hour to cyclic variables
#so that the model knows that hour 0 is close to hour 23, and month January is close to month December

#information about the week is missing



In [27]:
#add weekday information: feature from 1 to 7 indicating the day of the week
from dateutil.relativedelta import relativedelta

# Define the start date
start_date = datetime.datetime(2022, 1, 1)  # Assuming the year is 2022

# Add the "weekday" column
data_model['weekday'] = data_model.apply(lambda row: (start_date + relativedelta(days=row['day']-1, months=row['month']-1)).weekday() + 1, axis=1)

# Map weekday numbers to weekday names
weekday_names = np.array(range(1,8))
data_model['weekday'] = data_model['weekday'].map(lambda x: weekday_names[x-1])

data_model.head()



,month,day,hour,description,lamax,laeq,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,weekday
0,1,1,0,MP 03: Naamsestraat 62 Taste,60.322528,57.126833,0.00036,-18.197324,0.389565,6
1,1,1,0,MP 05: Calvariekapel KU Leuven,53.230972,49.987639,0.00036,-18.197324,0.389565,6
2,1,1,0,MP 06: Parkstraat 2 La Filosovia,53.666056,50.752000,0.00036,-18.197324,0.389565,6
3,1,1,0,MP 07: Naamsestraat 81,50.056861,47.440222,0.00036,-18.197324,0.389565,6
4,1,1,1,MP 03: Naamsestraat 62 Taste,53.033583,50.853806,0.00000,-16.227891,0.222602,6


In [32]:
#third of March: Tuesday
#print(data_model.loc[6000])

print("other day")
#17th of May: Tuesday
#print(data_model.loc[18000])

print("one of the last days, see if complete")
#first of December: Thursday
print(data_model.loc[50000])

#control is ok

other day
one of the last days, see if complete
month                               12
day                                  1
hour                                 0
description     MP 07: Naamsestraat 81
lamax                        50.872444
laeq                         48.564472
LC_DAILYRAIN                  0.001454
LC_WINDDIR                   -0.984568
LC_WINDSPEED                  0.031636
weekday                              4
Name: 50000, dtype: object
